In [1]:
from IPython.display import display
from bis2 import mlab
from bis import tir
from bis import itis
from bis import worms
from bis import tess
from bis import natureserve
from bson.objectid import ObjectId

In [2]:
bisDB = mlab.getDB("bis")
gapspecies = bisDB["gapspecies"]

In [3]:
thisRun = {}
thisRun["ITIS"] = False
thisRun["WoRMS"] = False
thisRun["TESS"] = True
thisRun["NatureServe"] = False

if thisRun["ITIS"]:
    itiscache = bisDB["itiscache"]
if thisRun["WoRMS"]:
    wormscache = bisDB["wormscache"]
if thisRun["TESS"]:
    tesscache = bisDB["tesscache"]
if thisRun["NatureServe"]:
    natureservecache = bisDB["natureservecache"]

for doc in gapspecies.find():
    if thisRun["ITIS"]:
        itisRecord = tir.baseTIRRegistration("gapspecies",ObjectId(doc["_id"]))
    if thisRun["WoRMS"]:
        wormsRecord = tir.baseTIRRegistration("gapspecies",ObjectId(doc["_id"]))
    if thisRun["TESS"]:
        tessRecord = tir.baseTIRRegistration("gapspecies",ObjectId(doc["_id"]))
    if thisRun["NatureServe"]:
        nsRecord = tir.baseTIRRegistration("gapspecies",ObjectId(doc["_id"]))
    
    # If the GAP species record has an explicit ITIS TSN, use it for ITIS and TESS searches
    if "ITIS_TSN" in doc:
        if thisRun["ITIS"]:
            itisRecord["searchURL"] = itis.getITISSearchURL(doc["ITIS_TSN"],False,False)
        if thisRun["TESS"]:
            tessRecord["searchURL"] = tess.getTESSSearchURL("TSN",doc["ITIS_TSN"])
    else:
        if thisRun["ITIS"]:
            itisRecord["searchURL"] = itis.getITISSearchURL(doc["scientificname"],False,False)
        if thisRun["TESS"]:
            tessRecord["searchURL"] = tess.getTESSSearchURL("SCINAME",doc["scientificname"])

    # Check to see if the caches currently have these records
    if thisRun["ITIS"]:
        itisCurrent = itiscache.find_one({"searchURL":itisRecord["searchURL"]},{"originCollection":1,"originID":1})
        if itisCurrent is None:
            itisCacheID = itiscache.insert_one(itisRecord).inserted_id
        else:
            itisCacheID = itisCurrent["_id"]
        if "itisCacheID" not in doc or doc["itisCacheID"] != itisCacheID:
            gapspecies.update_one({'_id':doc["_id"]}, {"$set":{"itisCacheID": itisCacheID}}, upsert=False)
    if thisRun["WoRMS"]:
        wormsRecord["searchURL"] = worms.getWoRMSSearchURL("ExactName",doc["scientificname"])
        wormsCurrent = wormscache.find_one({"searchURL":wormsRecord["searchURL"]},{"originCollection":1,"originID":1})
        if wormsCurrent is None:
            wormsCacheID = wormscache.insert_one(wormsRecord).inserted_id
        else:
            wormsCacheID = wormsCurrent["_id"]
        if "wormsCacheID" not in doc or doc["wormsCacheID"] != wormsCacheID:
            gapspecies.update_one({'_id':doc["_id"]}, {"$set":{"wormsCacheID": wormsCacheID}}, upsert=False)
    if thisRun["TESS"]:
        tessCurrent = tesscache.find_one({"searchURL":tessRecord["searchURL"]},{"originCollection":1,"originID":1})
        if tessCurrent is None:
            tessCacheID = tesscache.insert_one(tessRecord).inserted_id
        else:
            tessCacheID = tessCurrent["_id"]
        if "tessCacheID" not in doc or doc["tessCacheID"] != tessCacheID:
            gapspecies.update_one({'_id':doc["_id"]}, {"$set":{"tessCacheID": tessCacheID}}, upsert=False)
    if thisRun["NatureServe"]:
        nsRecord["searchURL"] = natureserve.getNatureServeNameSearchURL(doc["scientificname"])
        nsCurrent = natureservecache.find_one({"searchURL":nsRecord["searchURL"]},{"originCollection":1,"originID":1})
        if nsCurrent is None:
            natureServeCacheID = natureservecache.insert_one(nsRecord).inserted_id
        else:
            natureServeCacheID = nsCurrent["_id"]
        if "natureServeCacheID" not in doc or doc["natureServeCacheID"] != natureServeCacheID:
            gapspecies.update_one({'_id':doc["_id"]}, {"$set":{"natureServeCacheID": natureServeCacheID}}, upsert=False)

    print (doc["scientificname"])

    


Peromyscus leucopus
Sorex vagrans paludivagus
Aspidoscelis neotesselata
Thamnophis marcianus
Didelphis virginiana
Ursus arctos
Cophosaurus texanus
Pituophis catenifer
Deirochelys reticularia
Coluber flagellum
Petrosaurus mearnsi
Oxybelis aeneus
Graptemys pseudogeographica
Kinosternon arizonense
Sceloporus merriami
Sylvilagus bachmani
Rana luteiventris
Dipodomys ingens
Ammospermophilus harrisii
Pipilo crissalis
Peromyscus crinitus
Plethodon dorsalis
Plethodon grobmani
Thomomys umbrinus emotus
Tamias canipes
Odocoileus virginianus leucurus
Tamias sonomae
Phalacrocorax carbo
Ambystoma texanum
Megascops asio
Parula pitiayumi
Rallus limicola
Spermophilus brunneus
Thalasseus maximus
Rattus norvegicus
Urocyon littoralis santacruzae
Mephitis macroura
Chaetodipus hispidus
Vireo altiloquus
Marmota caligata
Urocyon littoralis santarosae
Puma yagouaroundi
Thryomanes bewickii
Dendroica coronata
Sigmodon arizonae plenus
Sceloporus grammicus
Sternotherus minor
Phenacomys ungava
Rattus rattus
Diadophi

In [13]:
for doc in gapspecies.find().distinct("ITIS_TSN"):
    print (doc)

180278
179932
683028
174144
179921
180543
173910
209400
173786
209320
173936
174258
173800
668672
173874
180123
550546
180240
180179
202307
180284
173653
208285
180230
180191
180699
180205
174715
173605
686658
178869
176221
180150
176922
180363
726907
180563
552483
179016
180139
726908
726257
178562
178891
180347
173871
173761
552492
180362
174158
174040
180294
683035
208992
179950
174218
173858
775125
173687
209451
174041
173693
176472
173594
173669
174240
174315
173505
173503
668325
173752
173725
178154
180133
179132
180381
174725
179022
179205
179233
550250
180203
668244
173612
174279
174024
174212
174311
208677
174318
176880
178911
173768
174248
174058
180182
175834
552518
180310
180276
179971
552512
173625
180324
174625
175089
178280
179952
180393
180604
774546
180286
180599
176517
179857
180701
180621
178973
775123
176837
179312
650445
178937
179811
177723
174824
174941
179371
177932
179340
176811
178257
177686
179232
173628
173666
554145
176828
552472
179402
177942
175590
726205

In [14]:
itiscache = bisDB["itiscache"]
for doc in gapspecies.find():
    if itiscache.find({"searchURL":{"$regex":"*"}})

http://services.itis.gov/?wt=json&rows=10&q=tsn:180278
http://services.itis.gov/?wt=json&rows=10&q=tsn:179932
http://services.itis.gov/?wt=json&rows=10&q=tsn:683028
http://services.itis.gov/?wt=json&rows=10&q=tsn:174144
http://services.itis.gov/?wt=json&rows=10&q=tsn:179921
http://services.itis.gov/?wt=json&rows=10&q=tsn:180543
http://services.itis.gov/?wt=json&rows=10&q=tsn:173910
http://services.itis.gov/?wt=json&rows=10&q=tsn:209400
http://services.itis.gov/?wt=json&rows=10&q=tsn:173786
http://services.itis.gov/?wt=json&rows=10&q=tsn:209320
http://services.itis.gov/?wt=json&rows=10&q=tsn:173936
http://services.itis.gov/?wt=json&rows=10&q=tsn:174258
http://services.itis.gov/?wt=json&rows=10&q=tsn:173800
http://services.itis.gov/?wt=json&rows=10&q=tsn:668672
http://services.itis.gov/?wt=json&rows=10&q=tsn:173874
http://services.itis.gov/?wt=json&rows=10&q=tsn:180123
http://services.itis.gov/?wt=json&rows=10&q=tsn:550546
http://services.itis.gov/?wt=json&rows=10&q=tsn:180240
http://ser